In [1]:
import sys
import csv
import xlsxwriter
import copy
import math
from functools import reduce

import numpy as np
import numpy.random as random #Random samples generator
from numpy.random import normal 
from random import shuffle
import scipy
import pandas as pd

import matplotlib.pyplot as plt #Para plotar o dataset
import matplotlib.patches as mpatches
import matplotlib.mlab as mlab #Para plotar pdf da gaussiana

from IPython.display import HTML, display

# 100 primos selecionados para gerar dados de teste
primes = [ 2,   3,   5,   7,   11,  13,  17,  19,  23,  29,
           31,  37,  41,  43,  47,  53,  59,  61,  67,  71, 
           73,  79,  83,  89,  97,  101, 103, 107, 109, 113, 
           127, 131, 137, 139, 149, 151, 157, 163, 167, 173,
           179, 181, 191, 193, 197, 199, 211, 223, 227, 229,
           233, 239, 241, 251, 257, 263, 269, 271, 277, 281,
           283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 
           353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 
           419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 
           467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 
           547 ]

In [2]:
def open_load_file(csvfilename):
    dataset = []
    
    with open(csvfilename, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        flag = True

        for row in spamreader:
            if not flag:
                rowdata = [float(row[i]) for i in range(len(row)-1)]
                rowdata.append(row[-1])
                dataset.append(rowdata)
            else:
                flag = False
    return dataset


In [3]:
def histogram_data(dataset, bins_number):
    dataset.sort()
    dataset_rounded = list(map((lambda x: round(x, 2)), dataset))

    min_value = min(dataset_rounded)
    step = (max(dataset_rounded)-min_value) * 1/(bins_number-1)
    sup_data_range = min_value + step

    frequency = [0]
    # usar como valor o meio do invertalo da classe
    data = [min_value + step/2]

    index = 0
    while( index < len(dataset_rounded) ):
        if dataset_rounded[index] < sup_data_range:
            frequency[-1] += 1
            index += 1
        else:
            frequency.append(0)
            data.append(sup_data_range + step * 0.5)
            sup_data_range += step
    return frequency, data

In [4]:
def divisors(n): 
    result = []
      
    # Note that this loop runs till square root 
    i = 1
    while i <= math.sqrt(n):     
        if (n % i == 0) : 
              
            # If divisors are equal, print only one 
            if (n / i == i) : 
                result.append(i), 
            else : 
                # Otherwise print both 
                result.append(i)
                result.append(math.floor(n/i))
        i = i + 1
    result.sort()
    return result

In [5]:
def percentil(dataset, percentil):
    index=(len(dataset) * (percentil* 0.01))
    if( len(dataset) == 0):
        return 0
    return round(dataset[int(index)],2)

In [6]:
def standard_desviation(dataset):
    media_local=mean(dataset)
    soma=0
    for i in range(0, len(dataset)):
        soma=(soma + ((dataset[i] - media_local)*(dataset[i] - media_local)))

    if len(dataset) < 2:
        return 0
    variance=(round(soma,2) * 1/(len(dataset) - 1.0))
    return math.sqrt(variance)

In [7]:
def mean(dataset):
    if len(dataset) == 0:
        return 0
    return reduce((lambda x,y: round(x + y,2)),dataset,0.) * 1/len(dataset)

In [8]:
def kurtosis(dataset): #curtose
    p25=percentil(dataset, 25)       
    p75=percentil(dataset, 75)
    
    p90=percentil(dataset, 90)
    p10=percentil(dataset, 10)
    
    cima=round((p75 - p25),2)
    baixo=round((p90 - p10),2)
    
    if( (cima == 0) or (baixo == 0)):
        return 0.0
    return (0.5) * (cima * 1/baixo)

In [9]:
def skew(dataset): #calcula assimetria
    media_local=mean(dataset)
    p50=percentil(dataset, 50)
    sd=standard_desviation(dataset)

    cima=(media_local - p50)
    if(cima == 0.0) or (sd == 0):
        return 0.0
    return 3.0 * (cima * 1/sd)

In [10]:
def discrete_mean(y,begin, end):   
    number_of_elements = end - begin
    if( number_of_elements == 0): 
        return 0.0
    return reduce((lambda x,y: x + y), y[begin:end],0.) * 1/number_of_elements

In [11]:
def weight_mean(x,y,begin, end):   
    sum_mean = 0.
    total_sample = 0

    for i in range(begin, end):
        sum_mean += x[i]*y[i]
        total_sample += y[i]

    if (total_sample == 0):
        return discrete_mean(x,begin,end)

    return sum_mean * 1/total_sample

In [12]:
def break_on_vales(data, frequency):
    status = [0 for i in range(len(frequency))]
    last_valley = 0
        
    # Classificando os pontos do meio
    for index in range(1,len(frequency)):
        if frequency[index-1] <= frequency[index]:
            if status[index-1] < 1:
                status[index] = 1
            elif status[index-1] == 1:
                status[index-1] = 0
                status[index] = 1
        elif frequency[index-1] > frequency[index]:
            if status[index-1] > -1:
                status[index] = -1
            elif status[index-1] == -1:
                status[index-1] = 0
                status[index] = -1
#         else:
#             if status[index - 1] != 0:
#                 status[index] = status[index - 1]*-1
#             else:
#                 status[index] = 0
                
    status[0]  = 0
    status[-1] = 0

    data_divisor = []
    # selecionando os vales que foram identificados
    for index in range(len(status)):
        if status[index] == -1:
            data_divisor.append(data[index])

    return data_divisor

def divisor(dataset, data_divisor):
    subdatasets = []
    next_dataset = dataset
    for elem in data_divisor:
        subdatasets.append(list(filter((lambda x: x <= elem),next_dataset)))
        next_dataset = list(filter((lambda x: x > elem),next_dataset))

    if len(next_dataset) > 1:
        subdatasets.append(next_dataset)

    return subdatasets

In [13]:
def entropic_indexes_calculation(dataset):
    data_mean = mean(dataset)
    data_kurtosis = kurtosis(dataset)
    data_skew = skew(dataset)

    return { 'mean': data_mean, 'kurtosis': data_kurtosis, 'skew': data_skew } 

def hepheastus(dataset):
    EIC = entropic_indexes_calculation(dataset)
    
    data_mean = EIC['mean']
    data_kurtosis = EIC['kurtosis']
    data_skew = EIC['skew']

    HFMS = []
        
    if math.fabs(data_skew) <= 1.:
        HFMS.append(dataset)            
    else:
        dataset_left  = list(filter((lambda x: x <= data_mean),dataset))
        dataset_right = list(filter((lambda x: x > data_mean),dataset))
        
        if len(dataset_left) > 1:
            HFMS += hepheastus(dataset_left)
        if len(dataset_right) > 1:
            HFMS += hepheastus(dataset_right)
    return HFMS 

In [14]:
def hercules(dataset,data, frequency, K, n):
    mean_frequency = []
    mean_data = []
    
    # Construcão da função de tendências
    for k in range(K):
        # print(k,k*n,(k+1)*n)
        # Média do número de leituras
        mean_frequency.append(discrete_mean(frequency,k*n, (k + 1)*n-1))
        # Média ponderada pelo número de leituras os dados 
        mean_data.append(weight_mean(data,frequency,k*n, (k + 1)*n-1))
    
    data_divisor = break_on_vales(mean_data, mean_frequency)
    
#     print(mean_frequency)
#     print(mean_data)
    
#     plt.hist(dataset, bins=K*n, color="#87CEFA", edgecolor='#101010')
#     for i in range(K-1):
#       plt.axvline(x=data[(i + 1)*n-1], color="#F3FC80")
#     plt.plot(mean_data, mean_frequency, marker="o", color="#F88B27")
#     for i in data_divisor:
#       plt.axvline(x=i, color="#73EC80")
#     plt.xlabel("Temperatura: Classes da Curva %d, Intervalos %d" % (K*n, K))
#     plt.ylabel("Número de leituras")
#     plt.show()
        
    return data_divisor

In [15]:
def maf(datasets):
    return [ mean(dataset) for dataset in datasets ]

# Aplicações

In [16]:
# 40°C to 65°C on overhead power line;
# 65°C to 75°C on the overhead power line;
def powerline(means):
    for elem in means:
        if elem < 65:
            return True
    return False

# 40°C to 144°C on the battery.
# Over 144 °C on the battery.
def battery(means):  
    for elem in means:
        if elem > 144:
            return False
    return True

def decisor(means):
    if(powerline(means) and battery(means)):
        return 'C1'
    elif(not powerline(means) and battery(means)):
        return 'C2'
    elif(powerline(means) and not battery(means)):
        return 'C3'
    elif(not powerline(means) and not battery(means)):
        return 'C4'
    

# Testes

## Gerador de Datasets

In [17]:
scenarios = ['C1','C2','C3','C4']

envs = { 'C1' : [[50.0,4.0],[92.0,18.0]],
         'C2' : [[85.0,2.0],[92.0,18.0]],
         'C3' : [[50.0,4.0],[155.0,2.0]],
         'C4' : [[85.0,2.0],[155.0,2.0]]  }

app1 = { 
    'C1': ['C1', 'C2'],
    'C2': ['C1', 'C2'],
    'C3': ['C3', 'C4'],
    'C4': ['C3', 'C4'] 
}

app2 = { 
    'C1': ['C1', 'C3'],
    'C2': ['C2', 'C4'],
    'C3': ['C1', 'C3'],
    'C4': ['C2', 'C4'] 
}

def generate_data(env, number_of_elements, state):
    data = []
    for app in env:
        mean = app[0]
        variance = app[1]
        for i in range(number_of_elements):
            data.append(round(random.normal(mean,variance),2))
    data.append(state)
    return data

def generate_dataset(positive,seed,number_of_experiments,number_of_elements):
    global env

    random.seed(seed)
    dataset = []

    for i in range(number_of_experiments):
        dataset.append(generate_data(env['C1'], number_of_elements, 'C1'))
        dataset.append(generate_data(env['C2'], number_of_elements, 'C2'))
        dataset.append(generate_data(env['C3'], number_of_elements, 'C3'))
        dataset.append(generate_data(env['C4'], number_of_elements, 'C4'))    

    filename = 'datasets/dataset-'+positive+'-'+str(seed)+'.csv'
    with open(filename, 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile)

        header = [i for i in range(number_of_elements*2)]
        header.append('result')
        spamwriter.writerow(header)

        shuffle(dataset)

        for data in dataset:
            spamwriter.writerow(data)
if (True == False):
    number_of_experiments = 100
    number_of_elements = 100

    for index, number in enumerate(primes):
        seed = number

        print("\rCompleted: %.4f" % ((index*100)/len(primes)), end="", flush=True)

        generate_dataset('C1',seed,number_of_experiments,number_of_elements)
        generate_dataset('C2',seed+1,number_of_experiments,number_of_elements)
        generate_dataset('C3',seed+2,number_of_experiments,number_of_elements)
        generate_dataset('C4',seed+3,number_of_experiments,number_of_elements)


    print("\nCompleted generated!", end="", flush=True)
else:
    print("\nAlready generated!", end="", flush=True)


Already generated!

## Calibrar parâmetros do Hercules. 

In [18]:
# def evaluate_result(positive, expect, result, tp, fp, tn, fn):
def evaluate_result(positive, expect, result, v):
    # No cenário em que eu testo a capacidade de assimilar o caso A
    
    # No caso em que testo valor A e passo o valor A
    if(positive == expect):
        # No caso em que espero o valor A e recebo o valor A
        if(expect == result):
            v['tp'] += 1
        # No caso em que espero o valor A e recebo o valor B
        else:
            v['fn'] += 1
    # No caso em que testo valor A e passo o valor B
    else:
        # espero o valor B e recebo o valor A
        if(positive == result):
            v['tn'] += 1
        # espero o valor B e recebo o valor B
        else:
            v['fp'] += 1
    return v

def evaluate_result_app(positive, expect, expect_value, result, v):
    if(expect_value in positive):
        if(result in expect):
            v['tp'] += 1
        else:
            v['fn'] += 1
    else:
        if(result in positive):
            v['tn'] += 1
        else:
            v['fp'] += 1
#     print(positive, expect, expect_value, result, v)
    return v

def evaluate_accuracy(v):
    return (v['tp']+v['fp']) * 1.0/(v['tp']+v['fp']+v['tn']+v['fn'])

In [19]:
def test_hercules(datasets, positive, N, K):
    global app1, app2
    
    valuesHer = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    valuesApp1Her = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    valuesApp2Her = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }

    for dataset in datasets:
        expect = dataset[-1]
        del dataset[-1]
        
        frequency,data = histogram_data(dataset,N)
        result_her = decisor(maf(divisor(dataset, hercules(dataset, data, frequency, K, math.floor(N/K)) )))        
        
        valuesHer = evaluate_result(positive, expect, result_her, valuesHer)
        valuesApp1Her = evaluate_result_app(app1[positive], app1[expect], expect, result_her, valuesApp1Her)
        valuesApp2Her = evaluate_result_app(app2[positive], app2[expect], expect, result_her, valuesApp2Her)
        

    acc = evaluate_accuracy(valuesHer)
    return acc


def calibrator(csvfilename, positive):
    dataset = open_load_file(csvfilename)
    n_bins = [i for i in range(10, len(dataset[0])-1)]
    steps = [divisors(i) for i in n_bins]
    func = [[-1 for j in step] for step in steps]

#     for i in range(len(n_bins)):
#         print("\rCompleted: %.4f" % ((i*100)/len(n_bins)), end="", flush=True)
#         for j in range(len(steps[i])):
#             dataset=open_load_file(csvfilename)
#             func[i][j]=test_hercules(dataset, positive, negative, n_bins[i], steps[i][j])

#     max_acc=0
#     min_n_bins=0
#     min_step=0
#     for i in range(len(n_bins)-1):
#         for j in range(len(steps[i])-1):
#             if(func[i][j] > max_acc):
#                 max_acc=func[i][j]
#                 min_n_bins=n_bins[i]
#                 min_step=steps[i][j]

    max_acc = test_hercules(dataset, positive, 10, 5)
    min_n_bins = 10
    min_step = 5

    print("\nMax Accuracy=%f, Bins=%d, Step=%d" % (max_acc, min_n_bins, min_step))
    return(min_n_bins, min_step)

if True:
    min_n_bins = 42
    min_step = 6
else:
    (min_n_bins, min_step) = calibrator('datasets/dataset-C1-101.csv', 'C1')
# (min_n_bins, min_step) = calibrator('datasets/dataset-C1-101.csv', 'C1')


## Executar casos de testes

In [20]:
def round_experiment(datasets, positive):
    global env,min_n_bins, min_step, app1, app2
    
    
        
    
    valuesMaf = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    rangeAcuracyMaf = []
    
    valuesApp1Maf = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    rangeAcuracyApp1Maf = []
    
    valuesApp2Maf = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    rangeAcuracyApp2Maf = []

    valuesHep = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    rangeAcuracyHep = []
    
    valuesApp1Hep = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    rangeAcuracyApp1Hep = []
    
    valuesApp2Hep = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    rangeAcuracyApp2Hep = []
    
    valuesHer = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    rangeAcuracyHer = []
    
    valuesApp1Her = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    rangeAcuracyApp1Her = []
    
    valuesApp2Her = { 'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0 }
    rangeAcuracyApp2Her = []
    
    for index, dataset in enumerate(datasets):
        expect = dataset[-1]        
        del dataset[-1]

        result_maf = decisor(maf([dataset]))
        result_hep = decisor(maf(hepheastus(dataset)))
        
        N = min_n_bins
        K = min_step
        n = math.floor(N/K)
        frequency,data = histogram_data(dataset,N)
        result_her = decisor(maf(divisor(dataset, hercules(dataset,data, frequency, K, n) )))

        valuesMaf = evaluate_result(positive, expect, result_maf, valuesMaf)
        valuesApp1Maf = evaluate_result_app(app1[positive], app1[expect], expect, result_maf, valuesApp1Maf)
        valuesApp2Maf = evaluate_result_app(app2[positive], app2[expect], expect, result_maf, valuesApp2Maf)
        
        valuesHep = evaluate_result(positive, expect, result_hep, valuesHep)
        valuesApp1Hep = evaluate_result_app(app1[positive], app1[expect], expect, result_hep, valuesApp1Hep)
        valuesApp2Hep = evaluate_result_app(app2[positive], app2[expect], expect, result_hep, valuesApp2Hep)
        
        valuesHer = evaluate_result(positive, expect, result_her, valuesHer)
        valuesApp1Her = evaluate_result_app(app1[positive], app1[expect], expect, result_her, valuesApp1Her)
        valuesApp2Her = evaluate_result_app(app2[positive], app2[expect], expect, result_her, valuesApp2Her)
        
    result = [ evaluate_accuracy(valuesMaf), evaluate_accuracy(valuesHep), evaluate_accuracy(valuesHer),
                evaluate_accuracy(valuesApp1Maf), evaluate_accuracy(valuesApp1Hep), evaluate_accuracy(valuesApp1Her),
                evaluate_accuracy(valuesApp2Maf), evaluate_accuracy(valuesApp2Hep), evaluate_accuracy(valuesApp2Her) ]

    return(result)
    
def run_drive(csvfilename, positive):
    dataset = []
    
    with open(csvfilename, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        flag = True # To ignore Header

        for index,row in enumerate(spamreader):
            if not flag:
                rowdata = [float(row[i]) for i in range(len(row)-1)]
                rowdata.append(row[-1])
                dataset.append(rowdata)
            else:
                flag = False
    
    return round_experiment(dataset, positive)

In [21]:
npAccC1 = []
npAccC2 = []
npAccC3 = []
npAccC4 = []

for index, number in enumerate(primes):
    seed = number
    print("\rCompleted: %.4f" % ((index*100)/len(primes)), end="", flush=True)
    npAccC1.append(run_drive('datasets/dataset-C1-'+str(seed+0)+'.csv','C1'))
    npAccC2.append(run_drive('datasets/dataset-C2-'+str(seed+1)+'.csv','C2'))
    npAccC3.append(run_drive('datasets/dataset-C3-'+str(seed+2)+'.csv','C3'))    
    npAccC4.append(run_drive('datasets/dataset-C4-'+str(seed+3)+'.csv','C4'))
    
npAccC1 = np.array(npAccC1).T
npAccC2 = np.array(npAccC2).T
npAccC3 = np.array(npAccC3).T
npAccC4 = np.array(npAccC4).T

print("\rCompleted: 100.0%", end="", flush=True)

Completed: 100.0%

In [22]:
def generate_interval(vector):
    mean = np.mean(vector)
    std = np.std(vector)
    interval = 1.96*std/len(vector)
    
    return (mean, std, interval)

def print_interval_acc(state, npAcc, index = 0):
    meanMafNpAcc, stdMafNpAcc, intervalMaf = generate_interval(npAcc[0 + index])
    meanHepNpAcc, stdHepNpAcc, intervalHep = generate_interval(npAcc[1 + index])
    meanHerNpAcc, stdHerNpAcc, intervalHer = generate_interval(npAcc[2 + index])
    
    return([state,
            str(round(meanMafNpAcc*100,4))+" %", "+/-"+str(round(intervalMaf*100,4))+" %",
            str(round(meanHepNpAcc*100,4))+" %", "+/-"+str(round(intervalHep*100,4))+" %",
            str(round(meanHerNpAcc*100,4))+" %", "+/-"+str(round(intervalHer*100,4))+" %"])

In [23]:
header = ["Caso(+/-): ",
            "Média Maf", "Intervalo Conf. Maf",
            "Média Hep", "Intervalo Conf. Hep",
            "Média Her", "Intervalo Conf. Her"]

table = [header,
            print_interval_acc("C1", npAccC1),
            print_interval_acc("C2", npAccC2),
            print_interval_acc("C3", npAccC3),
            print_interval_acc("C4", npAccC4)
        ]

df = pd.DataFrame(table)
display(HTML(df.to_html()))

table = [header,
            print_interval_acc("C1 - App1", npAccC1, 3),
            print_interval_acc("C2 - App1", npAccC2, 3),
            print_interval_acc("C3 - App1", npAccC3, 3),
            print_interval_acc("C4 - App1", npAccC4, 3)
        ]

df = pd.DataFrame(table)
display(HTML(df.to_html()))

table = [header,
            print_interval_acc("C1 - App2", npAccC1, 6),
            print_interval_acc("C2 - App2", npAccC2, 6),
            print_interval_acc("C3 - App2", npAccC3, 6),
            print_interval_acc("C4 - App2", npAccC4, 6)
        ]

df = pd.DataFrame(table)
display(HTML(df.to_html()))

,0,1,2,3,4,5,6
0,Caso(+/-):,Média Maf,Intervalo Conf. Maf,Média Hep,Intervalo Conf. Hep,Média Her,Intervalo Conf. Her
1,C1,75.0 %,+/-0.0 %,89.7723 %,+/-0.0289 %,99.4257 %,+/-0.0069 %
2,C2,25.0 %,+/-0.0 %,89.5693 %,+/-0.0228 %,99.4505 %,+/-0.0066 %
3,C3,75.0 %,+/-0.0 %,99.9728 %,+/-0.0015 %,99.9827 %,+/-0.0012 %
4,C4,75.0 %,+/-0.0 %,99.9802 %,+/-0.0015 %,99.9604 %,+/-0.0018 %


,0,1,2,3,4,5,6
0,Caso(+/-):,Média Maf,Intervalo Conf. Maf,Média Hep,Intervalo Conf. Hep,Média Her,Intervalo Conf. Her
1,C1 - App1,50.0 %,+/-0.0 %,99.9579 %,+/-0.0018 %,99.9158 %,+/-0.0027 %
2,C2 - App1,50.0 %,+/-0.0 %,99.9728 %,+/-0.0017 %,99.9505 %,+/-0.0021 %
3,C3 - App1,50.0 %,+/-0.0 %,99.9604 %,+/-0.0019 %,99.9183 %,+/-0.0029 %
4,C4 - App1,50.0 %,+/-0.0 %,99.9653 %,+/-0.0018 %,99.9406 %,+/-0.0021 %


,0,1,2,3,4,5,6
0,Caso(+/-):,Média Maf,Intervalo Conf. Maf,Média Hep,Intervalo Conf. Hep,Média Her,Intervalo Conf. Her
1,C1 - App2,50.0 %,+/-0.0 %,89.7723 %,+/-0.0286 %,99.4431 %,+/-0.0069 %
2,C2 - App2,50.0 %,+/-0.0 %,89.5767 %,+/-0.0228 %,99.4827 %,+/-0.0067 %
3,C3 - App2,50.0 %,+/-0.0 %,89.7104 %,+/-0.0333 %,99.4332 %,+/-0.0072 %
4,C4 - App2,50.0 %,+/-0.0 %,89.7327 %,+/-0.0281 %,99.4901 %,+/-0.0073 %
